# Creating a Service

### Introduction

Now that we understand a bit about how networking works in Kubernetes.  Let's create a service.  Remember that we use a service to allow access to a group of pods.   In this lesson, we'll create an *internal service*, which will allow a pod from one deployment to access pods from a different deployment.  Let's go.

### Getting setup

Now, we'll start from scratch by first creating our api-deployment which will setup our api pods.  Then we'll move onto the service.

Our `api-deployment` config looks like the following:

```yaml
# api-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-deployment
spec:
  replicas: 2
  selector:
    matchLabels:
      component: api
  template:
    metadata:
      labels:
        component: api
    spec:
      containers:
        - name: flask-api
          image: jek2141/flask_api
          ports:
          - containerPort: 3000
```

And after running `kubectl apply -f api-deployment.yaml`, we can see that the container port 3000 on the pod is exposed -- and this is where the flask api is running.

### Creating our Service

Now, to make the pods in this deployment more easily accessible, we'll create our service.  

<img src="./updated-ports.jpg" width="60%">

Ok, let's take a look at the config file for the corresponding service.

```yaml
apiVersion: v1
kind: Service
metadata:
  name: api-cluster-ip-service
spec:
  type: ClusterIP
  selector: 
    component: api
  ports: 
    - port: 5000
      targetPort: 5000
```

The first few lines are pretty typical: we specify:
* `apiVersion: v1`: We are working with the core Kubernetes library 
* `kind: Service`: This is the component we are creating
* `metadata: ` Here, we just specify the name.

Let's more spend some more time discussing the rest of the config file.

* **ClusterIP**
    * So under `spec`, we specify a type of `ClusterIP`.  This is the type of service we use when allowing only requests internal to our cluster -- that is, requests from another pod -- to access our service.  We'll handle external requests later.
    
* **Selector**
    * With selector, we specify the pods to forward the requests to.  Here we'll forward requests to those pods with the label of `component: api`.
    
* **Ports** 
    * Then, under ports we'll specify the port that the service will be listening on, `port`, and then port that it forwards requests to, `targetPort`.  So here we'll make a request to the service on port 3000, and service will forward the request to one of the pods with the label `api` on the targetPort of 3000.

> The port is the port that the api-service listens for requests on, and the targetPort is the port that it forwards requests to on the node.  We can see this with another look to our diagram.

<img src="./updated-ports.jpg" width="60%">

### Trying it Out

Ok, so now we'll deploy the service.  

`kubectl apply -f api-service.yaml`

If we run `kubectl describe service` we can see some of the setup.

<img src="./describe-service.png" width="90%">

Under `IP` we see the virtual ip of the service.  And then, under endpoints, we see the ip addresses of pods  that it points to.

Ok, now we can only make requests to this service from inside the cluster.  So let's create deploy another set of pods from which we can make a request to the api service.  

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: flask-deployment
spec:
  replicas: 2
  selector:
    matchLabels:
      component: flask
  template:
    metadata:
      labels:
        component: flask
    spec:
      containers:
        - name: flask-api
          image: jek2141/apartments_api
          ports:
          - containerPort: 3000
```

`kubectl apply -f flask-deployment.yaml`

> This also runs a flask app, but we're just going to use ensure we can make a request to the api service from a pod.

Then we'll connect to one of the flask deployment pods, and see if we can make a request to our service.

`kubectl exec -it flask-deployment-5fffd7fc6c-9mr9r`

> Once connected to our flask deployment pod, try to make a request to the service via `wget api-deployment-service:5000/restaurants`.

<img src="./connect-to-service.png" width="90%">

It looks like that worked!  Notice that we made the request just by specifying the name of the service, `api-deployment-service:5000/restaurants`.  This routed a request to one of our pods and the pod sent back the response to the api-deployment pod, as we can see above.

### Summary

In this lesson, we saw how to both create and connect to our service.  We created a clusterIP service, and then specified both the selector -- that is the pods to forward the requests onto, and the ports.  With the ports we specified the port of the service to connect to and the `targetPort` of the pods.  

Then we connected to a pod in a different deployment and made a request to our new service using the pattern of `wget domain-name:port_number/url`.